* I experiment with getting the "anomaly score" using Isolation forests, and add that as a feature and see if it improves our supervised fraud prediction task. 
    * It does!
    
    
* Isolation forest and other semisupervised approaches:
    * https://www.kaggle.io/svf/1100683/56c8356ed1b0a6efccea8371bc791ba7/__results__.html#Tree-based-techniques

* Kernel uses the imbalanced-learn (imblearn) library for boilerplate under (or over) sampling.
    * pip install imblearn

In [1]:
import pandas as pd
from imblearn.under_sampling import RandomUnderSampler
from sklearn.ensemble import RandomForestClassifier, IsolationForest

from xgboost import XGBClassifier
from sklearn.model_selection import StratifiedKFold , train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import roc_curve, auc , confusion_matrix
import numpy as np
np.random.seed(42)
from sklearn.metrics import roc_curve, auc

### Build the data set from file

In [2]:
credit_cards=pd.read_csv('../input/creditcard.csv')
columns=credit_cards.columns
# The labels are in the last column ('Class').

In [3]:
import numpy as np
credit_cards['Amount'] = np.log(credit_cards['Amount'] + 1)
credit_cards['Time'] = np.log(credit_cards['Time'] + 1)

In [4]:
features_train = credit_cards

## Isolation forest (anomaly detection)
* https://www.kaggle.io/svf/1100683/56c8356ed1b0a6efccea8371bc791ba7/__results__.html#Tree-based-techniques
) 
* __contamination__ = % of anomalies expected

In [5]:
model = IsolationForest(random_state=42,  max_samples=0.96, bootstrap=True, n_estimators=90,contamination= 0.002)
model.fit(features_train)

In [6]:
print(model.decision_function(credit_cards[credit_cards['Class'] == 0].values).mean())

* View anomaly score vs each class:
    * (Commented out)

In [7]:
# print(model.decision_function(credit_cards[credit_cards['Class'] == 0].drop('Class', axis=1).values).mean())
# print(model.decision_function(credit_cards[credit_cards['Class'] == 1].drop('Class', axis=1).values).mean())

In [8]:
credit_cards["isolation_score"] = model.decision_function(credit_cards.drop('Class', axis=1).values)

### Build train and test sets (20% of data reserved to test set)
* Run with or without dropping the __isolation_score__ and compare results! 
* Be sure to keep seed fixed for consistency
* downsampling method may affect results

In [ ]:
features=credit_cards.drop(['Class'],axis=1)
labels=credit_cards['Class']

features_train, features_test, labels_train, labels_test = train_test_split(features, 
                                                                            labels, 
                                                                            test_size=0.2, 
                                                                            random_state=0)

### Downsample the majority class in the train set
* Could use other downsampling methods such as centroids, clusters etc'

In [ ]:
labels_train.shape

In [ ]:
# oversampler=SMOTE(random_state=0)
sampler = RandomUnderSampler(random_state=0,ratio=0.006)
os_features,os_labels=sampler.fit_sample(features_train,labels_train)

In [ ]:
# verify new data set is balanced
len(os_labels[os_labels==1])

In [ ]:
os_labels.shape

# New

### Perform training of the random forest using the (sampled) train set

#### Model without isolation forest feature:

In [ ]:
clf=RandomForestClassifier(random_state=0,class_weight="balanced",n_estimators=300,criterion='entropy')
clf.fit(os_features.drop("isolation_score",axis=1),os_labels)

In [ ]:
# perform predictions on test set
actual=labels_test
predictions=clf.predict(features_test)

In [ ]:
confusion_matrix(actual,predictions)

In [ ]:
false_positive_rate, true_positive_rate, thresholds = roc_curve(actual, predictions)
roc_auc = auc(false_positive_rate, true_positive_rate)
print (roc_auc)

# Rerun with Isolation forest/anomaly metafeature:





clf=RandomForestClassifier(random_state=0,class_weight="balanced",n_estimators=300,criterion='entropy')
clf.fit(os_features,os_labels)


actual=labels_test
predictions=clf.predict(features_test)

confusion_matrix(actual,predictions)

false_positive_rate, true_positive_rate, thresholds = roc_curve(actual, predictions)
roc_auc = auc(false_positive_rate, true_positive_rate)
print (roc_auc)

### We improved our results by using  the  isolation forest score as a new metafeature! 

Final results locally:

* With isolation forest feat: 91.08  AUC
* Without: 0.90587 AUC

    * Sample class ratio: 0.006